## Limpieza de Datos: Microdatos sobre defunciones de INEGI

Autor: Fernanda Muñoz Arroyo

Versión: 11/12/25

In [84]:
# IGNORAR
# ver los valores únicos para CAUSA_DEF y entender qué codigos se están usando en la base
# df["CAUSA_DEF"].str[:3].value_counts().head(20)
# df[df["CAUSA_DEF"].str[:1].isin(["X","Y"])][["CAUSA_DEF"]].head(20)
# usamos el incorrecto para homicidios, y filtramos las defunciones neonatales, por eso nos salía todo concentrado en ese grupo de edad


In [78]:
import pandas as pd

# cargamos el dbf de defunciones
ruta = "defunciones_base_datos_2022_dbf"
archivo_def = f"{ruta}/DEFUN22.dbf"  # ajusta segun el año

df = pd.DataFrame(DBF(archivo_def, load=True))

print("Columnas disponibles:", df.columns.tolist())


# filtramos presuntos homicidios según CIE-10
# CAUSA_DEF contiene códigos tipo 'X950', 'Y040'
# https://ais.paho.org/classifications/chapters/pdf/volume1.pdf
# Para clasificar homicidio, usamos prefijos CIE-10: X85–Y09 (agresiones)
# Tomamos solo los primeros 3 caracteres del string 

df["causa3"] = df["CAUSA_DEF"].str[:3]

# Lista de prefijos válidos para homicidio (modificarse con un experto)
homicidio_prefix = []
homicidio_prefix.extend([f"W{n:02d}" for n in range(32, 34)])  
homicidio_prefix.extend([f"W{n:02d}" for n in range(50, 51)])  
homicidio_prefix.extend([f"X{n:02d}" for n in range(85, 99)])  
# Filtrar solo homicidios
df_h = df[df["causa3"].isin(homicidio_prefix)].copy()




Columnas disponibles: ['ENT_REGIS', 'MUN_REGIS', 'TLOC_REGIS', 'LOC_REGIS', 'ENT_RESID', 'MUN_RESID', 'TLOC_RESID', 'LOC_RESID', 'ENT_OCURR', 'MUN_OCURR', 'TLOC_OCURR', 'LOC_OCURR', 'CAUSA_DEF', 'COD_ADICIO', 'LISTA_MEX', 'SEXO', 'ENT_NAC', 'AFROMEX', 'CONINDIG', 'LENGUA', 'CVE_LENGUA', 'NACIONALID', 'NACESP_CVE', 'EDAD', 'SEM_GEST', 'GRAMOS', 'DIA_OCURR', 'MES_OCURR', 'ANIO_OCUR', 'DIA_REGIS', 'MES_REGIS', 'ANIO_REGIS', 'DIA_NACIM', 'MES_NACIM', 'ANIO_NACIM', 'COND_ACT', 'OCUPACION', 'ESCOLARIDA', 'EDO_CIVIL', 'TIPO_DEFUN', 'OCURR_TRAB', 'LUGAR_OCUR', 'PAR_AGRE', 'ASIST_MEDI', 'CIRUGIA', 'NATVIOLE', 'NECROPSIA', 'USONECROPS', 'ENCEFALICA', 'DONADOR', 'SITIO_OCUR', 'COND_CERT', 'DERECHOHAB', 'EMBARAZO', 'REL_EMBA', 'HORAS', 'MINUTOS', 'CAPITULO', 'GRUPO', 'LISTA1', 'GR_LISMEX', 'AREA_UR', 'EDAD_AGRU', 'COMPLICARO', 'DIA_CERT', 'MES_CERT', 'ANIO_CERT', 'MATERNAS', 'ENT_OCULES', 'MUN_OCULES', 'LOC_OCULES', 'RAZON_M', 'DIS_RE_OAX']


In [79]:
# Contar cuántos están en cada bloque 1xxx, 2xxx, 3xxx, ...
def bloque(v):
    try:
        v = int(v)
    except:
        return "NA"
    if 1000 <= v <= 1999:
        return "1xxx"
    if 2000 <= v <= 2999:
        return "2xxx"
    if 3000 <= v <= 3999:
        return "3xxx"
    if 4000 <= v <= 4999:
        return "4xxx"
    if 5000 <= v <= 5999:
        return "5xxx"
    if 6000 <= v <= 6999:
        return "6xxx"
    return "other"

df_h["bloque_EDAD"] = df_h["EDAD"].apply(bloque)
print("\nConteo por bloque EDAD:")
print(df_h["bloque_EDAD"].value_counts())



Conteo por bloque EDAD:
bloque_EDAD
4xxx    25080
3xxx       12
2xxx        4
1xxx        2
Name: count, dtype: int64


In [80]:
# las edades estan en un formato de 4 digitos, y para X95 (CIE-10) es decir, homicidio por arma de fuego, esas son las edades. 
df[df["CAUSA_DEF"].str.startswith("X95")][["EDAD"]].sample(20)

,EDAD
701518,4031
241777,4040
806178,4043
282890,4040
15767,4046
482678,4045
271247,4998
67199,4030
11273,4998
763610,4045


In [81]:
# sorted(df["EDAD"].dropna().unique())[:50], acá vemos los valores únicos de la columna de edad
# la edad viene en un formato que desconocía, acá está la funcion provisional de transformacion 

def edad_provisional(valor):
    """
    Extrae los últimos dos dígitos del código EDAD de INEGI
    para obtener una edad aproximada en años.
    
    Ejemplo:
    - 1025 -> 25
    - 1987 -> 87
    - 4032 -> 32 (en teoría horas, pero lo tratamos como 32 años)
    
    la codificación original está desbalanceadam, los homicidios aparecen con 4xxx. 
    """
    if pd.isna(valor):
        return None

    # Convertir a string
    s = str(int(valor))

    # Últimos dos dígitos
    return int(s[-2:])

# lo aplicamos a la columna correspondiente
df_h["EDAD_ANIOS"] = df_h["EDAD"].apply(edad_provisional)



In [82]:

# clasificamos en grupos de edad
df_h["grupo_edad"] = pd.cut(
    df_h["EDAD_ANIOS"],
    bins=[-1, 9, 19, 35, 59, 200],
    labels=[
        "Infancias (0-9)",
        "Adolescentes (10-19)",
        "Jóvenes (20-35)",
        "Adultos (36-59)",
        "Adultos mayores (60+)"
    ]
)

# clasificamos sexo
df_h["sexo2"] = df_h["SEXO"].map({1: "Hombre", 2: "Mujer"})


# agregación por entidad de ocurrencia ENT_OCURR:

# total de homicidios por entidad
total = df_h.groupby("ENT_OCURR").size().rename("total_presuntos_homicidios")

# Totales por sexo
sexo = df_h.pivot_table(index="ENT_OCURR",
                        columns="sexo2",
                        values="CAUSA_DEF",
                        aggfunc="count").fillna(0)

# Totales por grupo de edad
edades = df_h.pivot_table(index="ENT_OCURR",
                          columns="grupo_edad",
                          values="CAUSA_DEF",
                          aggfunc="count").fillna(0)


# unimos todo en un df
resultado = pd.concat([total, sexo, edades], axis=1).reset_index()


resultado.head()

/var/folders/dn/8pfwn8qj1q5b6xfw68s4ccmm0000gn/T/ipykernel_21919/1289958239.py:30: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  edades = df_h.pivot_table(index="ENT_OCURR",


,ENT_OCURR,total_presuntos_homicidios,Hombre,Mujer,Infancias (0-9),Adolescentes (10-19),Jóvenes (20-35),Adultos (36-59),Adultos mayores (60+)
0,01,53,47,6,1,1,34,15,2
1,02,2159,1901,226,1,90,829,837,402
2,03,34,27,7,0,1,20,10,3
3,04,64,59,5,0,3,31,27,3
4,05,62,51,11,0,5,34,20,3


In [83]:
# exportar en el mismo directorio
resultado.to_csv("hom22.csv", index=False)